In [1]:
from models.model_wrapper import VLM
llava_model = VLM(model='LLAVA')

/home2/anony_ai/miniconda3/envs/myenv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


🚀 Loading LLAVA model...


Loading checkpoint shards: 100%|██████████| 3/3 [00:28<00:00,  9.65s/it]
Some parameters are on the meta device because they were offloaded to the cpu.


✅ Model loaded!


In [9]:
resp = llava_model.run_inference(
    "./VisualGenome/VG_100K/2.jpg", 
    "Based on the image, Fill the blank with SINGLE VERB. Man ____ street..", 
    level=3
)
print(resp)

{'answer': 'Walking', 'first_word_probs': {'crossing': 0.04418598952788155, 'walking': 0.2015957347114039}}


In [3]:
from models.model_wrapper import VLM
blipi_model = VLM(model='InstructBLIP')

🚀 Loading InstructBLIP model...


Loading checkpoint shards: 100%|██████████| 2/2 [01:35<00:00, 47.73s/it]


✅ Model loaded!


In [4]:
resp = blipi_model.run_inference(
    "./VisualGenome/VG_100K/2.jpg", 
    "Based on the image, Fill it with appropriate option. Man ____ street (walking, running, crossing, watching, looking). Only answer option within the given set, should not create variations of the following.", 
    level=3
)
print(resp)

{'answer': 'crossing', 'first_word_probs': {'crossing': 1.0}}


: 

In [ ]:
from models.model_wrapper import VLM
blip2_model = VLM(model='BLIP2')

In [24]:
resp = blip2_model.run_inference(
    "./VisualGenome/VG_100K/2.jpg", 
    "Answer only 'yes' or 'no'. If you're unsure, output 'no'. Is there a black building?", 
    level=1
)
print(resp)

{'answer': "I don't see a black building", 'first_word_probs': {'yes': 0.3841719478368759, 'no': 0.17784660309553146}}


In [1]:
from models.model_wrapper import VLM
gemma_model = VLM(model='InstructBLIP')

/home2/anony_ai/miniconda3/envs/myenv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


🚀 Loading InstructBLIP model...


Loading checkpoint shards: 100%|██████████| 2/2 [02:20<00:00, 70.41s/it]


✅ Model loaded!


In [2]:
resp = gemma_model.run_inference(
    "./VisualGenome/VG_100K/2.jpg", 
    "Answer only 'yes' or 'no'. If you're unsure, output 'no'. Is there a green car?", 
    level=1
)
print(resp)

RuntimeError: Input type (torch.cuda.HalfTensor) and weight type (torch.HalfTensor) should be the same

In [30]:
from tqdm import tqdm

for q in tqdm(queries):
    question = q["question"]
    parsed = f'Instruction: {q['instruction']}, {question}'
    response = run_inference_llava(get_image(q['image_id']), parsed)
    q["llava_response"] = response

# Save results to new file
with open("llava_responses.json", "w") as f:
    json.dump(queries, f, indent=2)

print("Done! Answers written to llava_responses.json 🎉")

100%|██████████| 10/10 [00:28<00:00,  2.89s/it]

Done! Answers written to llava_responses.json 🎉


In [2]:
import os
import json
import random
from tqdm import tqdm

# Get image path from image_id
def get_image(image_id):
    filename = f"{image_id}.jpg"
    path1 = os.path.join("./VisualGenome/VG_100K/", filename)
    path2 = os.path.join("./VisualGenome/VG_100K_2/", filename)

    if os.path.exists(path1):
        return path1
    elif os.path.exists(path2):
        return path2
    else:
        raise FileNotFoundError(f"Image {filename} not found in either folder.")

# Main function with system prompt support
def run_vlm_on_queries(query_file_path, output_path=None, system_prompt=None, inference_func=None, level=1):

    results = []

    with open(query_file_path, 'r') as f:
        queries = json.load(f)

    with open("./assets/final_cleaned_relations.json", 'r') as f:
        relations = json.load(f)    
    # queries = random.sample(queries, 100)

    original_system_prompt = system_prompt
    for q in tqdm(queries):
        image_id = q['image_id']
        query = q['question']
        gt_answer = q['answer']

        try:
            image_path = get_image(image_id)
        except FileNotFoundError as e:
            print(e)
            continue
        
        try:
            # if level == 3:
            #     formatted_relations = random.sample(relations, min(10, len(relations)))
            #     formatted_relations.append(gt_answer)
            #     formatted_relations = ", ".join(formatted_relations)
            #     system_prompt = system_prompt.replace("<actions>", formatted_relations)
            
            if level == 4:
                for option in q['options']:
                    system_prompt += f"""\n {option},"""
                
                system_prompt += f""" {q['answer']}"""

            pred_object = inference_func(image_path, f"{system_prompt}\n USER:{query}", level=level)
            pred_answer = pred_object['answer']
        except Exception as e:
            print(f"Inference failed for image {image_id}: {e}")
            pred_answer = None

        system_prompt = original_system_prompt
        results.append({
            "image_id": image_id,
            "query": query,
            "ground_truth": gt_answer,
            "prediction": pred_answer,
            "word_probs": pred_object["first_word_probs"],
        })

    if output_path:
        with open(output_path, 'w') as f:
            json.dump(results, f, indent=2)

    return results

In [1]:
from models.model_wrapper import VLM
llava_model = VLM(model='LLAVA')

/home2/anony_ai/miniconda3/envs/myenv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


🚀 Loading LLAVA model...


Loading checkpoint shards: 100%|██████████| 3/3 [00:57<00:00, 19.17s/it]
Some parameters are on the meta device because they were offloaded to the cpu.


✅ Model loaded!


In [3]:
# LEVEL 1

system_prompt = """
    Answer ONLY 'yes' or 'no' based on: 
    1. 'yes' - Object is clearly and unambiguously visible 
    2. 'no' - Object is absent or uncertain.
    Never explain.
"""

run_vlm_on_queries(
    query_file_path="./queries/queries_level_1.json", 
    output_path="./results/blip2_level_1.json", 
    system_prompt=system_prompt, 
    inference_func=blip2_model.run_inference, 
    level=1
)

  0%|          | 0/1055 [00:00<?, ?it/s]

100%|██████████| 1055/1055 [03:12<00:00,  5.47it/s]


[{'image_id': 615,
  'query': 'Can you see man in this picture?',
  'ground_truth': 'Yes',
  'prediction': 'No',
  'word_probs': {'yes': 0.5094880312681198, 'no': 0.3096214234828949}},
 {'image_id': 615,
  'query': 'Is there trash can in the image?',
  'ground_truth': 'Yes',
  'prediction': 'Yes',
  'word_probs': {'yes': 0.744194358587265, 'no': 0.13734273612499237}},
 {'image_id': 615,
  'query': 'Can you see shawl in this picture?',
  'ground_truth': 'Yes',
  'prediction': 'Yes',
  'word_probs': {'yes': 0.7008896470069885, 'no': 0.2991103455424309}},
 {'image_id': 615,
  'query': 'Does the image contain any bag?',
  'ground_truth': 'Yes',
  'prediction': 'aye',
  'word_probs': {'yes': 0.18023721873760223, 'no': 0.48900168389081955}},
 {'image_id': 615,
  'query': 'Can you see stripe in this picture?',
  'ground_truth': 'Yes',
  'prediction': 'Yes',
  'word_probs': {'yes': 0.5730619579553604, 'no': 0.25226929783821106}},
 {'image_id': 1523,
  'query': 'Is there phone in the image?',
 

In [3]:
# LEVEL 2

system_prompt = """
    Answer ONLY 'yes' or 'no' based on: 
    1. 'yes' - Object is clearly and unambiguously visible 
    2. 'no' - Object is absent or uncertain.
    Never explain.
"""

run_vlm_on_queries(
    query_file_path="./queries/queries_level_2.json", 
    output_path="./results/blip2_level_2.json", 
    system_prompt=system_prompt, 
    inference_func=blip2_model.run_inference, 
    level=2
)

100%|██████████| 1051/1051 [03:28<00:00,  5.03it/s]


[{'image_id': 615,
  'query': 'Is there drum in the image?',
  'ground_truth': 'No',
  'prediction': 'YES',
  'word_probs': {'yes': 0.6674026362721861, 'no': 0.29641812294721603}},
 {'image_id': 615,
  'query': 'Is there concert in the image?',
  'ground_truth': 'No',
  'prediction': 'yes',
  'word_probs': {'yes': 0.5952256321907043, 'no': 0.22157791256904602}},
 {'image_id': 615,
  'query': 'Does the image contain any wallet?',
  'ground_truth': 'No',
  'prediction': 'No',
  'word_probs': {'yes': 0.21297310292720795, 'no': 0.5467083156108856}},
 {'image_id': 615,
  'query': 'Is there book in the image?',
  'ground_truth': 'No',
  'prediction': 'yes',
  'word_probs': {'yes': 0.6554327607154846, 'no': 0.17246390879154205}},
 {'image_id': 615,
  'query': 'Does the image contain any pocket?',
  'ground_truth': 'No',
  'prediction': 'No',
  'word_probs': {'yes': 0.17741148173809052, 'no': 0.47375163435935974}},
 {'image_id': 1523,
  'query': 'Does the image contain any cup?',
  'ground_tru

In [20]:
# LEVEL 3

system_prompt = """
    Analyze the image and identify the action (verb) between the subject and object. 
    1. Focus only on visible, physical actions (e.g., 'holding', 'pushing', 'watching').

  adjusting, advertising, approaching, attaching, bending, biting, blocking, blowing, brushing, building, buying,
  carrying, casting, catching, chasing, checking, cleaning, climbing, connecting, containing, controlling, cooking, 
  covering, crashing, creating, crossing, cutting, decorating, displaying, dividing, doing, dragging, drinking, driving, eating, entering,
  exiting, facing, feeding, filming, fixing, flying, folding, following, forming, getting, giving, 
  going, grabbing, grazing, growing, guarding, guiding, hanging, having, helping, herding, hitting, 
  holding, hugging, illuminating, indicating, jumping, kicking, kissing, laying, leading, leaving, licking, lifting, loading, looking, lying, making, marking,
  milking, moving, opening, operating, passing, performing, petting, photographing, pitching, playing, 
  pointing, pouring, preparing, protecting, pulling, pushing, putting, racing, raising, reading, releasing, resting, riding, rolling, rowing, rubbing, running, scratching,
  selling, separating, serving, shading, shaking, sharing, shining, showing, sitting, skiing, slicing, 
  smoking, sniffing, spraying, spreading, standing, steering, sticking, stopping, stretching, supporting, 
  surfing, swimming, swinging, taking, talking, teaching, throwing, tossing, touching, towing, training, 
  tying, using, walking, waving, wearing, working

  The output should ONLY be SINGLE WORD from the following list.
  OUTPUT ONLY SINGLE WORD, NOTHING ELSE
"""

run_vlm_on_queries(
    query_file_path="./queries/queries_level_3.json", 
    output_path="./results/blip2_level_3_CoT.json", 
    system_prompt=system_prompt, 
    inference_func=blip2_model.run_inference, 
    level=3
)

100%|██████████| 935/935 [08:23<00:00,  1.86it/s]


[{'image_id': 615,
  'query': 'what is the relationship between man and jacket?',
  'ground_truth': 'wearing',
  'prediction': 'ad',
  'word_probs': {}},
 {'image_id': 1523,
  'query': 'what is the relationship between light and keyboard?',
  'ground_truth': 'hitting',
  'prediction': '"Assistant"',
  'word_probs': {}},
 {'image_id': 1901,
  'query': 'what is the relationship between person and boat?',
  'ground_truth': 'standing',
  'prediction': '1.',
  'word_probs': {}},
 {'image_id': 1901,
  'query': 'what is the relationship between man and shirt?',
  'ground_truth': 'wearing',
  'prediction': 'a shirt',
  'word_probs': {}},
 {'image_id': 3963,
  'query': 'what is the relationship between woman and shirt?',
  'ground_truth': 'wearing',
  'prediction': 'a woman is sitting on a table.',
  'word_probs': {}},
 {'image_id': 3963,
  'query': 'what is the relationship between human and shirt?',
  'ground_truth': 'wearing',
  'prediction': 'a white shirt',
  'word_probs': {}},
 {'image_id

: 

In [6]:
# LEVEL 4

system_prompt = """
    Analyze the image and identify the action (verb) between the subject and object. 
    Strictly follow these rules:
    1. Focus only on visible, physical actions (e.g., 'holding', 'pushing', 'watching').
    2. The output should ONLY be SINGLE WORD from the following list.
    OUTPUT ONLY SINGLE WORD, NOTHING ELSE
"""

LEVEL = 4
run_vlm_on_queries(
    query_file_path=f"./queries/queries_level_{LEVEL}.json", 
    output_path=f"./results/blip2_level_{LEVEL}.json", 
    system_prompt=system_prompt, 
    inference_func=blip2_model.run_inference, 
    level=LEVEL,
)

  0%|          | 0/352 [00:00<?, ?it/s]

100%|██████████| 352/352 [03:20<00:00,  1.76it/s]


[{'image_id': 615,
  'query': 'Based on the image, what is the relationship between man and jacket? Choose only from the options provided',
  'ground_truth': 'wearing',
  'prediction': 'HELP',
  'word_probs': {}},
 {'image_id': 1523,
  'query': 'Based on the image, what is the relationship between light and keyboard? Choose only from the options provided',
  'ground_truth': 'hitting',
  'prediction': 'A person can be a soldier, but not a soldier-in-training.',
  'word_probs': {}},
 {'image_id': 1901,
  'query': 'Based on the image, what is the relationship between man and shirt? Choose only from the options provided',
  'ground_truth': 'wearing',
  'prediction': 'a man is standing in front of a blue shirt',
  'word_probs': {}},
 {'image_id': 1901,
  'query': 'Based on the image, what is the relationship between person and boat? Choose only from the options provided',
  'ground_truth': 'standing',
  'prediction': 'a boat',
  'word_probs': {}},
 {'image_id': 3963,
  'query': 'Based on th

### CLIP model (image captioning)

In [ ]:
import clip
import torch
import os
from PIL import Image

# os.environ["TORCH_HOME"] = "./clip_cache"

# Set device
device = "cuda" if torch.cuda.is_available() else "cpu"

# Load the model
model, preprocess = clip.load("ViT-B/32", device=device)

def clip_query(image_path, options):
    """Query CLIP with multiple text options for a given image"""
    # Preprocess image
    image = preprocess(Image.open(image_path)).unsqueeze(0).to(device)

    # Tokenize text options
    text = clip.tokenize(options).to(device)

    # Get model predictions
    with torch.no_grad():
        image_features = model.encode_image(image)
        text_features = model.encode_text(text)

        # Normalize features
        image_features /= image_features.norm(dim=-1, keepdim=True)
        text_features /= text_features.norm(dim=-1, keepdim=True)

        # Compute similarity
        similarity = (image_features @ text_features.T).squeeze(0)
        probs = similarity.softmax(dim=-1).cpu().numpy()

    return {
        "answer": options[probs.argmax()],
        "confidence": float(probs.max()),
        "all_probs": {opt: float(prob) for opt, prob in zip(options, probs)}
    }

# Example usage
result = clip_query(
    "./VisualGenome/VG_100K/2.jpg",
    options=["street", "mountains", "himalaya"]
)
print(f"Detected: {result['answer']} (Confidence: {result['confidence']:.2f})")


Detected: street (Confidence: 0.35)


## Attributes

In [3]:
import json
import os
from tqdm import tqdm

def get_image_path(image_id):
    filename = f"{image_id}.jpg"
    vg1 = os.path.join("./VisualGenome/VG_100K", filename)
    vg2 = os.path.join("./VisualGenome/VG_100K_2", filename)
    if os.path.exists(vg1):
        return vg1
    elif os.path.exists(vg2):
        return vg2
    else:
        raise FileNotFoundError(f"{filename} not found in VG folders.")

def evaluate_model_on_queries(queries_path, output_path, system_prompt=None):
    with open(queries_path, "r") as f:
        queries_by_type = json.load(f)

    results = []
    
    for attr_type, queries in queries_by_type.items():
        print(f"Evaluating queries for attribute type: {attr_type}")
        for q in tqdm(queries):
            try:
                image_path = get_image_path(q["image_id"])
                response = llava_model.run_inference(image_path, f"SYSTEM:{system_prompt}\n USER:{q["query"]}")
                results.append({
                    "image_id": q["image_id"],
                    "attribute_type": attr_type,
                    "object": q["object"],
                    "attribute": q["attribute"],
                    "query": q["query"],
                    "ground_truth": q["answer"],
                    "prediction": response
                })
            except Exception as e:
                print(f"Skipping {q['image_id']} due to error: {e}")
                continue

    with open(output_path, "w") as f:
        json.dump(results, f, indent=2)
    print(f"Saved results to {output_path}")

# Run evaluation
evaluate_model_on_queries(
    queries_path="./queries/queries_attr.json",
    output_path="./results/llava_level_attr.json",
    system_prompt="""You are a visual attribute extractor. Given an input query about an object's attribute (color, material, state, etc.), respond with ONLY THE EXACT SINGLE WORD that answers the query.  
**Rules:**
1. Output must be **one word only** (e.g., "red", "wooden", "broken").
2. Never explain, describe, or add punctuation.
3. If uncertain, output "unknown".

**Examples:**
Input: "What color is the apple?"
"red"

Input: "What material is the table made of?"
"wood"
"""
)

Evaluating queries for attribute type: color


100%|██████████| 618/618 [17:08<00:00,  1.66s/it]


Evaluating queries for attribute type: material


100%|██████████| 43/43 [01:14<00:00,  1.74s/it]


Evaluating queries for attribute type: condition


100%|██████████| 12/12 [00:26<00:00,  2.18s/it]


Evaluating queries for attribute type: shape


100%|██████████| 9/9 [00:15<00:00,  1.75s/it]

Saved results to ./results/llava_level_attr.json
